In [ ]:
import pandas as pd 
pd.set_option('display.max_columns', None)
import re 

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')


original[['milage', 'price']] = original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

In [ ]:
original.head()

In [ ]:
train.head(2)

In [ ]:
train.info()

In [ ]:
train['fuel_type'].isnull().sum()

In [ ]:
train[train['fuel_type'].isnull()].sample(5)

In [ ]:
train[(train['brand'].isin(['Tesla','Audi','Porsche']) == False) & (train['fuel_type'].isnull())]

In [ ]:
train['fuel_type'].unique()

In [ ]:
train[(train['fuel_type'].isin(['–']) == True)]

In [ ]:
train[(train['fuel_type'].isin(['not supported']) == True)]

In [ ]:
train['fuel_type'] = train['fuel_type'].replace('not supported', 'Electric')
train['fuel_type'] = train['fuel_type'].replace('–', 'Gasoline')
train['fuel_type'] = train['fuel_type'].fillna('Electric')

In [ ]:
test['fuel_type'] = test['fuel_type'].replace('not supported', 'Electric')
test['fuel_type'] = test['fuel_type'].replace('–', 'Gasoline')
test['fuel_type'] = test['fuel_type'].fillna('Electric')

In [ ]:
train['fuel_type'].unique()


In [ ]:
test['fuel_type'].unique()

In [ ]:
train.columns

In [ ]:
a = train['accident'].isna().sum()

In [ ]:
( a / len(train['accident']) ) *100

In [ ]:
train['accident'].unique()

In [ ]:
train['accident'] = train['accident'].fillna('None reported')

In [ ]:
test['accident'] = test['accident'].fillna('None reported')

In [ ]:
original['accident'] = original['accident'].fillna('None reported')

In [ ]:
train[train['accident'].isna()]

In [ ]:
train['accident'] = train['accident'].fillna('No')

In [ ]:
test['accident'] = train['accident'].fillna('No')

In [ ]:
train[train['clean_title'].isna()].head(3)

In [ ]:
train['clean_title'] = train['clean_title'].fillna('No')

In [ ]:
test['clean_title'] = test['clean_title'].fillna('No')

In [ ]:
original['clean_title'] = original['clean_title'].fillna('No')

In [ ]:
train['clean_title'].isna()

In [ ]:
train['clean_title'].unique()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
# Define a function to extract engine attributes
def extract_engine_attributes(engine_str):
    horsepower = re.search(r'(\d+\.\d+)HP|\d+\.\d+', engine_str)
    displacement = re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', engine_str)
    return horsepower.group(1) if horsepower else '',\
           displacement.group(1) if displacement else ''

# Apply the function to create new columns
train[['horsepower', 'engine_displacement']] = train['engine'].apply(extract_engine_attributes).apply(pd.Series)
test[['horsepower', 'engine_displacement']] = test['engine'].apply(extract_engine_attributes).apply(pd.Series)

In [ ]:
train.sample(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
categorical_columns=[feature for feature in train.columns if train[feature].dtype=='O']
numerical_columns=[feature for feature in train.columns if train[feature].dtype!='O']

In [ ]:
label=LabelEncoder()
categorical_columns = train.select_dtypes(include ='object').columns
for column in categorical_columns:
    train[column] = train[column].astype(str)
    test[column] = test[column].astype(str)
    test[column] = label.fit_transform(test[column])
    train[column] = label.fit_transform(train[column])

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split , cross_val_score
import warnings
from sklearn.metrics import mean_squared_error

In [ ]:
X = train.drop(columns=['price'], axis =1)
y = train['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state = 1
)

In [ ]:
'''def objective_lgbm(trial):

    lgbm_params = {
    'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
    'num_leaves': trial.suggest_int('num_leaves', 115, 250),
    'max_depth': trial.suggest_int('max_depth', 25, 40),
    'cat_smooth': trial.suggest_int('cat_smooth', 50, 200),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.01, log = True),
    'subsample': trial.suggest_float('subsample', 0.7, 0.9,log = True),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.9, log = True),
    'min_split_gain': trial.suggest_float('min_split_gain', 1e-4, 0.8, log = True),
    'min_child_weight': trial.suggest_int('min_child_weight', 30, 90),
    'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 1e-5,log = True),
    'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 1e-5,log = True),
    #'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 12.0),
    'max_bin': trial.suggest_int('max_bin', 500, 900),
    }
    

    # Fit the model
    model_lgbm = LGBMRegressor(
        **lgbm_params, random_state= 1, loss_function="RMSE", verbose=-1
    )

    model_lgbm.fit(X_train, y_train)

    # Predict and calculate mean squared error score
    y_pred = model_lgbm.predict(X_test)

    return mean_squared_error(y_test, y_pred, squared = False)'''

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
import optuna

In [ ]:
'''study = optuna.create_study(direction='minimize')
study.optimize(objective_lgbm, n_trials=50)'''

In [ ]:
'''best_params = study.best_params
print("Best parameters:", best_params)'''

In [ ]:
lgbm_params = {
    'n_estimators': 1247,
    'num_leaves': 123,
    'max_depth': 28,
    'cat_smooth': 143,
    'learning_rate': 0.0027129249008214056,
    'subsample': 0.7977379135998542,
    'colsample_bytree': 0.4588317424928748,
    'min_split_gain': 0.003110663752733955,
    'min_child_weight': 85,
    'lambda_l2': 3.3613296039345123e-08,
    'lambda_l1': 8.445494309658136e-08,
    'max_bin' :777
}

In [ ]:
model_1 = LGBMRegressor(**lgbm_params)
model_1.fit(X_train, y_train)

In [ ]:
y_pred_lgb = model_1.predict(test)

In [ ]:
y_pred_lgb

In [ ]:
ids =  sample_submission['id']

In [ ]:
sub = pd.DataFrame({'id' : ids, 
                   'price' : y_pred_lgb})

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv', index = False)